In [ ]:
"""
創建 queue:
    List How many queues in SQS service 
    Create a queue
    List How many queues in SQS service 

發送訊息至 queue:
    List messages in queue
    Send a message into queue
    List messages in queue

測試可見性逾時:
    List messages in queue (the message invisible)
        wait 10 seconds
    List messages in queue (the message visible)

需使用receipt_handle 來刪除訊息:
    Get a message and it's receipt_handle from queue 
    Delete message from queue
        wait 10 second
    List messages in queue

刪除 queue:
    Delete the queue

"""

In [ ]:
!pip install boto3 awscli

In [ ]:
# Get the service resource

import boto3

sqs = boto3.resource('sqs', endpoint_url='http://localstack-main:4566')

print(sqs)

In [ ]:
# List How many queues in SQS service 

queue_iterator = sqs.queues.all()

for queue in queue_iterator:
    print(queue)
    print(queue.url)
    print(queue.attributes)

In [ ]:
# Create a queue

my_queue = sqs.create_queue(QueueName='my_queue')
print(my_queue)

In [ ]:
# List How many queues in SQS service 

queue_iterator = sqs.queues.all()

for queue in queue_iterator:
    print(queue)
    print(queue.url)
    print(queue.attributes)

In [ ]:
# List messages in queue

list_of_message = my_queue.receive_messages(
    MaxNumberOfMessages=10,  # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=3,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)
)

print(list_of_message)

for message in list_of_message:
    print(message.message_id)
    print(message.body)

In [ ]:
# Send a message into queue

# MessageBody (string) [REQUIRED] The maximum string size is 256 KB.

response = my_queue.send_message(MessageBody='this is a message, please take it..')

response

In [ ]:
# List messages in queue

list_of_message = my_queue.receive_messages(
    MaxNumberOfMessages=10,  # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=3,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)
)

print(list_of_message)

for message in list_of_message:
    print(message.message_id)
    print(message.body)


In [ ]:
"""
剛才的操作有時會取得不到訊息
    等一段時間後又取得到
    原因是可見性逾時

此步驟調整可見性逾時為 10 秒做測試
List messages in queue (the message invisible)
wait 10 seconds
List messages in queue (the message visible)

https://docs.aws.amazon.com/zh_tw/AWSSimpleQueueService/latest/SQSDeveloperGuide/sqs-visibility-timeout.html
可見性逾時 VisibilityTimeout: 
    接收消息時設定，訊息的預設可見性逾時為 30 秒。下限為 0 秒。上限為 12 小時。
"""


list_of_message = my_queue.receive_messages(
    MaxNumberOfMessages=10,  # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=0,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)
    VisibilityTimeout=10,  # 可見性逾時預設 30 秒。下限： 0 秒，上限： 12 小時。
)

print(list_of_message)

for message in list_of_message:
    print(message.message_id)
    print(message.body)


In [ ]:
"""
Get a message from queue 
Delete message from queue
    wait 10 second
List messages in queue

"""

# 取得一筆訊息
list_of_message = my_queue.receive_messages(
    MaxNumberOfMessages=1,  # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=0,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)
    VisibilityTimeout=10,  # 可見性逾時預設 30 秒。下限： 0 秒，上限： 12 小時。
)

print(list_of_message)

if len(list_of_message) > 0:
    print(list_of_message[0].receipt_handle)



In [ ]:
# 使用ReceiptHandle 刪除此訊息，每次接收訊息 ReceiptHandle 都會不一樣

response = queue.delete_messages(
    Entries=[
        {
            'Id': 'something_you_can_distinguish',  # 因為此動作為批次刪除訊息，為了區分不同訊息處理結果，可自定義 Id 來區分
            'ReceiptHandle': '上一步驟取得的 receipt_handle'
        },
        
    ]
)

response

In [ ]:
# 再次收取訊息

list_of_message = my_queue.receive_messages(
    MaxNumberOfMessages=10,  # Retrieves one or more messages (up to 10)
    WaitTimeSeconds=0,  # 設為 0 時: 短輪詢；大於 0 時: 長輪詢(上限20秒)
    VisibilityTimeout=10,  # 可見性逾時預設 30 秒。下限： 0 秒，上限： 12 小時。
)

print(list_of_message)

for message in list_of_message:
    print(message.message_id)
    print(message.body)


In [ ]:
# Deletes the queue specified by the QueueUrl, regardless of the queue's contents. 

response = my_queue.delete()

print(response)